# Image viewer

This notebook is for inspecting timelapse microscopy data, with associated sinhgle-cell labels and tracks, showing the infection of human macrophages with Mycobacterium Tuberculosis (Mtb), acquired on an Opera Phenix confocal microscope. 

In [1]:
import napari
import os, glob
from macrohet import dataio #, visualise, notify
import numpy as np
import os
import re
import numpy as np
import cv2
import btrack
import zarr


### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [2]:
%%time
expt_ID = 'ND0005'

base_dir = f'/mnt/SYNO/macrohet_syno/data/{expt_ID}/'
# base_dir = f'/mnt/DATA/macrohet/{expt_ID}/'

metadata_fn = glob.glob(os.path.join(base_dir, 'acquisition/Images/Index*xml'))[0]
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata

Reading metadata XML file...


0it [00:00, ?it/s]

Extracting metadata complete!
CPU times: user 3.23 s, sys: 317 ms, total: 3.55 s
Wall time: 4.34 s


,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0303K1F1P1R1,Ok,r03c03f01p01-ch1sk1fk1fl1.tiff,3,3,1,1,0,1,1,...,0,0.135257497,0,2024-10-06T18:14:34.337+01:00,640,706,40,1.1,0.2,"[[0.999498,0,0,-8.1],[0,-0.999498,0,-1.0],[0,0..."
1,0303K1F1P1R2,Ok,r03c03f01p01-ch2sk1fk1fl1.tiff,3,3,1,1,0,2,1,...,0,0.135257497,0,2024-10-06T18:14:34.57+01:00,488,522,40,1.1,0.1,"[[0.999498,0,0,-8.1],[0,-0.999498,0,-1.0],[0,0..."
2,0303K1F1P2R1,Ok,r03c03f01p02-ch1sk1fk1fl1.tiff,3,3,1,2,0,1,1,...,2E-06,0.135259494,0,2024-10-06T18:14:34.913+01:00,640,706,40,1.1,0.2,"[[0.999498,0,0,-8.1],[0,-0.999498,0,-1.0],[0,0..."
3,0303K1F1P2R2,Ok,r03c03f01p02-ch2sk1fk1fl1.tiff,3,3,1,2,0,2,1,...,2E-06,0.135259494,0,2024-10-06T18:14:35.133+01:00,488,522,40,1.1,0.1,"[[0.999498,0,0,-8.1],[0,-0.999498,0,-1.0],[0,0..."
4,0303K1F1P3R1,Ok,r03c03f01p03-ch1sk1fk1fl1.tiff,3,3,1,3,0,1,1,...,4E-06,0.135261506,0,2024-10-06T18:14:35.477+01:00,640,706,40,1.1,0.2,"[[0.999498,0,0,-8.1],[0,-0.999498,0,-1.0],[0,0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31093,0404K192F9P1R2,Ok,r04c04f09p01-ch2sk192fk1fl1.tiff,4,4,9,1,191,2,1,...,0,0.135189295,57306.39,2024-10-07T10:10:41.33+01:00,488,522,40,1.1,0.1,"[[0.999498,0,0,-8.1],[0,-0.999498,0,-1.0],[0,0..."
31094,0404K192F9P2R1,Ok,r04c04f09p02-ch1sk192fk1fl1.tiff,4,4,9,2,191,1,1,...,2E-06,0.135191306,57306.39,2024-10-07T10:10:41.677+01:00,640,706,40,1.1,0.2,"[[0.999498,0,0,-8.1],[0,-0.999498,0,-1.0],[0,0..."
31095,0404K192F9P2R2,Ok,r04c04f09p02-ch2sk192fk1fl1.tiff,4,4,9,2,191,2,1,...,2E-06,0.135191306,57306.39,2024-10-07T10:10:41.893+01:00,488,522,40,1.1,0.1,"[[0.999498,0,0,-8.1],[0,-0.999498,0,-1.0],[0,0..."
31096,0404K192F9P3R1,Ok,r04c04f09p03-ch1sk192fk1fl1.tiff,4,4,9,3,191,1,1,...,4E-06,0.135193303,57306.39,2024-10-07T10:10:42.237+01:00,640,706,40,1.1,0.2,"[[0.999498,0,0,-8.1],[0,-0.999498,0,-1.0],[0,0..."


### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [3]:
metadata_path = glob.glob(os.path.join(base_dir, 'acquisition/Assaylayout/*.xml'))[0]
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,replicate_number=False)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
assay_layout

Reading metadata XML file...
Extracting metadata complete!


Strain Compound Concentration ConcentrationEC
Row Column                                              
3   1         UNI     CTRL             0             EC0
    2         UNI     CTRL             0             EC0
    3          WT     CTRL             0             EC0
    4          WT     CTRL             0             EC0
    5          WT      PZA            60            EC50
    6          WT      PZA            60            EC50
    7          WT      RIF           0.1            EC50
    8          WT      RIF           0.1            EC50
    9          WT      INH          0.04            EC50
    10         WT      INH          0.04            EC50
    11         WT      BDQ          0.02            EC50
    12         WT      BDQ          0.02            EC50
4   3          WT     CTRL             0             EC0
    4          WT     CTRL             0             EC0
    5          WT      PZA           400            EC99
    6          WT      PZA           400            EC99
    7          WT      RIF             2            EC99
    8          WT      RIF             2            EC99
    9          WT      INH             2            EC99
    10         WT      INH             2            EC99
    11         WT      BDQ           2.5            EC99
    12         WT      BDQ           2.5            EC99
5   3         RD1     CTRL             0             EC0
    4         RD1     CTRL             0             EC0
    5         RD1      PZA            60            EC50
    6         RD1      PZA            60            EC50
    7         RD1      RIF           0.1            EC50
    8         RD1      RIF           0.1            EC50
    9         RD1      INH          0.04            EC50
    10        RD1      INH          0.04            EC50
    11        RD1      BDQ          0.02            EC50
    12        RD1      BDQ          0.02            EC50
6   3         RD1     CTRL             0             EC0
    4         RD1     CTRL             0             EC0
    5         RD1      PZA           400            EC99
    6         RD1      PZA           400            EC99
    7         RD1      RIF             2            EC99
    8         RD1      RIF             2            EC99
    9         RD1      INH             2            EC99
    10        RD1      INH             2            EC99
    11        RD1      BDQ           2.5            EC99
    12        RD1      BDQ           2.5            EC99

### Load using Zarr

In [4]:
expt_ID

'ND0005'

In [5]:
acq_ID = (3, 4)

In [6]:
image_dir = os.path.join(base_dir, f'acquisition/zarr/{acq_ID}.zarr')

zarr_group = zarr.open(image_dir, mode='r')

In [6]:
%%time
images = zarr_group.images[:,1,1,...]
images.shape

CPU times: user 9.75 s, sys: 4.12 s, total: 13.9 s
Wall time: 3.67 s


(150, 6048, 6048)

In [9]:
zarr_group.images

<zarr.core.Array '/images' (192, 2, 3, 6048, 6048) uint16 read-only>

In [7]:
%%time
images_max_proj = np.max(zarr_group.images, axis = 2)

CPU times: user 1min 35s, sys: 1min 9s, total: 2min 44s
Wall time: 8min 21s


In [ ]:
images_max_proj.shape

In [7]:
images.shape

(150, 6048, 6048)

#### Load downscaled version from mp4 videos

In [11]:
mp4_video_height

0

In [8]:
mp4_path = glob.glob(f'/mnt/SYNO/videos/macrohet_videos/{expt_ID}/{expt_ID}_{acq_ID}*')[0]

In [14]:
mp4_path#


'/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/mp4/(3, 4).mp4'

In [ ]:
/mnt/SYNO/macrohet_syno/data/ND0005/acquisition/mp4/(3, 4)

In [28]:
mp4_video_width / 1200

1.6

In [44]:
# Load the MP4 video to get its width and height (assuming square frames)
mp4_path = f'/mnt/SYNO/macrohet_syno/data/{expt_ID}/acquisition/mp4/{acq_ID}/{acq_ID}.mp4'

# Open the video using OpenCV
cap = cv2.VideoCapture(mp4_path)

# Get video dimensions (assume width = height)
mp4_video_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
mp4_video_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Compute scale factor to rescale the tracks to fit the MP4 dimensions
scale_factor =  mp4_video_width / 1200

# Scale tuple for napari tracks
track_scale = (1, scale_factor, scale_factor)  # Assuming time is not scaled

# Print the computed scale tuple
print(f"Scale for napari tracks: {track_scale}")

# Code to load MP4 video frames as a stack of images
frames = []
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    # Convert the frame to grayscale (if needed)
    # frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Uncomment for grayscale
    frames.append(frame)

# Convert list of frames to a NumPy array (stack of images)
images_mp4 = np.stack(frames, axis=0)

# Release the video capture object
cap.release()

Scale for napari tracks: (1, 1.6, 1.6)


In [42]:
from tqdm.auto import tqdm

In [45]:
# Initialize an empty list to store the rescaled masks
rescaled_segmentations = []

# Iterate through the segmentation stack, rescaling each frame
for i in tqdm(range(segmentation.shape[0]), total = segmentation.shape[0], desc="Rescaling segmentations"):
    # Extract the 2D mask for the current time point
    mask = segmentation[i]
    
    # Rescale the mask to match the MP4 video dimensions
    rescaled_mask = cv2.resize(mask, (mp4_video_width, mp4_video_height), interpolation=cv2.INTER_NEAREST)
    
    # Append the rescaled mask to the list
    rescaled_segmentations.append(rescaled_mask)

# Convert the list of rescaled masks back into a NumPy array (3D stack)
rescaled_stack = np.stack(rescaled_segmentations, axis=0)

# Now `rescaled_stack` contains the resized segmentations that match the MP4 video dimensions
print(f"Rescaled segmentation stack shape: {rescaled_stack.shape}")


Rescaling segmentations:   0%|          | 0/192 [00:00<?, ?it/s]

Rescaled segmentation stack shape: (192, 1920, 1920)


# Launch napari

In [ ]:
viewer = napari.Viewer(title = f'{expt_ID, acq_ID}, max_proj')


viewer.add_image(images_max_proj,
                 channel_axis = 1, 
                 colormap=['magenta', 'green'],
                 blending = 'additive', 
                 contrast_limits=[[350, 1000],[0, 7000]]
                )

In [17]:
viewer = napari.Viewer(title = f'{expt_ID, acq_ID}, max_proj')


viewer.add_image(images_mp4
                )

<Image layer 'images_mp4' at 0x7f5106f42620>

In [18]:
with btrack.io.HDF5FileHandler(os.path.join(f'/mnt/SYNO/macrohet_syno/data/{expt_ID}/labels/cpv3/{acq_ID}.h5'), #macrohet_seg_model 
                                           'r', 
                                           obj_type='obj_type_1'
                                           ) as reader:
                segmentation = reader.segmentation
                tracks = reader.tracks

[INFO][2024/10/22 02:33:15 pm] Opening HDF file: /mnt/SYNO/macrohet_syno/data/ND0005/labels/cpv3/(3, 4).h5...
[INFO][2024/10/22 02:33:39 pm] Loading segmentation (192, 6048, 6048)
[INFO][2024/10/22 02:33:39 pm] Loading tracks/obj_type_1
[INFO][2024/10/22 02:33:39 pm] Loading LBEP/obj_type_1
[INFO][2024/10/22 02:33:40 pm] Loading objects/obj_type_1 (76971, 5) (76971 filtered: None)
[INFO][2024/10/22 02:33:41 pm] Closing HDF file: /mnt/SYNO/macrohet_syno/data/ND0005/labels/cpv3/(3, 4).h5


### Add tracks

In [ ]:
filtered_tracks = [track for track in tracks if len(track) > 35]

In [47]:
# recoloured_seg = btrack.utils.update_segmentation(segmentation, filtered_tracks, scale = (5.04, 5.04))
recoloured_seg = btrack.utils.update_segmentation(rescaled_stack, tracks, scale = (1.6, 1.6))

In [23]:
# napari_tracks, _, _ = btrack.utils.tracks_to_napari(filtered_tracks, ndim = 2)
napari_tracks, _, _ = btrack.utils.tracks_to_napari(tracks, ndim = 2)

In [30]:
# tracks_scale = (1, 5.04, 5.04)
viewer.add_tracks(napari_tracks, scale=track_scale, name = 'tracks')

<Tracks layer 'tracks [1]' at 0x7f506536ee90>

Traceback (most recent call last):
  File "/home/dayn/miniconda3/envs/godspeed/lib/python3.10/site-packages/vispy/util/event.py", line 469, in _invoke_callback
    cb(event)
  File "/home/dayn/miniconda3/envs/godspeed/lib/python3.10/site-packages/napari/_vispy/camera.py", line 244, in viewbox_mouse_event
    super().viewbox_mouse_event(event)
  File "/home/dayn/miniconda3/envs/godspeed/lib/python3.10/site-packages/vispy/scene/cameras/perspective.py", line 233, in viewbox_mouse_event
    self._update_rotation(event)
  File "/home/dayn/miniconda3/envs/godspeed/lib/python3.10/site-packages/vispy/scene/cameras/arcball.py", line 65, in _update_rotation
    Quaternion(*_arcball(self._event_value, wh)) *
  File "/home/dayn/miniconda3/envs/godspeed/lib/python3.10/site-packages/vispy/scene/cameras/arcball.py", line 101, in _arcball
    x, y = xy
ValueError: too many values to unpack (expected 2)

During handling of the above exception, another exception occurred:

Traceback (most recent call la

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
File ~/miniconda3/envs/godspeed/lib/python3.10/site-packages/napari/_qt/threads/status_checker.py:100, in StatusChecker.calculate_status(self=<napari._qt.threads.status_checker.StatusChecker object>)
     97     return
     99 try:
--> 100     res = viewer._calc_status_from_cursor()
        viewer = Viewer(camera=Camera(center=(486.12053448959847, 716.9323638321471, 1152.4819589408314), zoom=0.48356271157375297, angles=(0.393957687301852, 4.388437407546903, 129.56980303361428), perspective=0.0, mouse_pan=True, mouse_zoom=True), cursor=Cursor(position=(569.0103203910519, 851.1492648342622, 870.9827342919884), scaled=True, style=<CursorStyle.STANDARD: 'standard'>, size=1.0), dims=Dims(ndim=3, ndisplay=3, order=(0, 1, 2), axis_labels=('0', '1', '2'), rollable=(True, True, True), range=(RangeTuple(start=0.0, stop=191.0, step=1.0), RangeTupl

Traceback (most recent call last):
  File "/home/dayn/miniconda3/envs/godspeed/lib/python3.10/site-packages/napari/_qt/threads/status_checker.py", line 86, in run
    self.calculate_status()
  File "/home/dayn/miniconda3/envs/godspeed/lib/python3.10/site-packages/napari/_qt/threads/status_checker.py", line 110, in calculate_status
    self.status_and_tooltip_changed.emit(res)
UnboundLocalError: local variable 'res' referenced before assignment


### Add segmentation

In [48]:
viewer.add_labels(rescaled_stack) #segmentation)
viewer.add_labels(recoloured_seg)

<Labels layer 'recoloured_seg' at 0x7f508fa98ee0>

In [28]:
viewer.add_image(images_max_proj[-5],
                 channel_axis = 0, 
                 colormap=['green', 'magenta'],
                 blending = 'additive', 
                 contrast_limits=[[0, 2400],[0, 1000]]
                )

[<Image layer 'Image [4]' at 0x7fa481516460>,
 <Image layer 'Image [5]' at 0x7fa4823087f0>]

In [ ]:
viewer.camera.reset()

In [14]:
visualise.highlight_cell(126, viewer, filtered_tracks, scale_factor=5.02)

<Points layer 'cell 126' at 0x7f248be4fa60>

In [20]:
for layer in viewer.layers:
    viewer.layers[layer.name].scale = (10, 1, 1)

In [21]:
viewer.layers[2].name

'filtered'

In [22]:
viewer.layers[2].scale = (10,5.04, 5.04)

# Saving out as a list of JPG files for SAM2 testing

In [40]:
from tqdm.auto import tqdm
from PIL import Image
import os
import cv2
import numpy as np

# Assuming `images_max_proj` is a numpy array of images with shape (n_images, channels, height, width)
output_dir = f"/mnt/SYNO/macrohet_syno/data/{expt_ID}/acquisition/jpegs/{acq_ID}"  # Directory to save the images
os.makedirs(output_dir, exist_ok=True)

# Ensure the MP4 output directory exists
mp4_dir = f"/mnt/SYNO/macrohet_syno/data/{expt_ID}/acquisition/mp4/{acq_ID}"
os.makedirs(mp4_dir, exist_ok=True)

# Set the MP4 output path
output_video_path = os.path.join(mp4_dir, f"{acq_ID}.mp4")

# Initialize video writer for the MP4 file using OpenCV
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # MP4 codec
fps = 10  # Set frames per second
downscaled_size = (1920, 1920)  # New size for downscaling to 1000x1000

video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, downscaled_size)

# Contrast limits for both channels
contrast_limits = [[350, 1000], [0, 7000]]

# Function to apply contrast stretching
def apply_contrast_limits(image, vmin, vmax):
    image = np.clip(image, vmin, vmax)
    image = (image - vmin) / (vmax - vmin) * 255
    return image.astype(np.uint8)

# Iterate over each image in both channels
for idx, (img_ch1, img_ch2) in enumerate(tqdm(zip(images_max_proj[:, 1, ...], images_max_proj[:, 0, ...]), total = 192, desc="Saving images and creating video")):
    # Apply contrast limits to both channels
    img_ch1_adjusted = apply_contrast_limits(img_ch1, *contrast_limits[1])
    img_ch2_adjusted = apply_contrast_limits(img_ch2, *contrast_limits[0])
    
    # Create an RGB image where channel 1 is green and channel 0 is magenta
    img_rgb = np.zeros((img_ch1.shape[0], img_ch1.shape[1], 3), dtype=np.uint8)
    img_rgb[..., 1] = img_ch1_adjusted  # Green channel for ch1
    img_rgb[..., 0] = img_ch2_adjusted  # Red channel for ch0 (magenta = red + blue)
    img_rgb[..., 2] = img_ch2_adjusted  # Blue channel for ch0

    # Downscale the image to 1000x1000 px
    img_downscaled = cv2.resize(img_rgb, downscaled_size, interpolation=cv2.INTER_AREA)
    
    # Save each downscaled image as a 5-digit frame number in jpg format
    frame_filename = os.path.join(output_dir, f"{idx:05d}.jpg")
    cv2.imwrite(frame_filename, img_downscaled)

    # Write the current downscaled frame to the video
    video_writer.write(cv2.imread(frame_filename))

# Release the video writer to finalize the video file
video_writer.release()

Saving images and creating video:   0%|          | 0/192 [00:00<?, ?it/s]